# Introduction to NLP Fundamentals in Tensorflow

Natural Language Process (NLP) is the technology used to aid computers to undestand human's natural language. (could be sequences text or speech)

Another common term for NLP problem is sequence to sequence problems (seq2seq).

In [5]:
# Import libraries
import random
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import train_test_split

print(f'Tensorflow Version: {tf.__version__}')

Tensorflow Version: 2.12.0


## Check for GPU

In [6]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-829757bc-1b96-e473-2914-b0ad47ca18e7)


## Get the helper function and import some functions

In [7]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_histories

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster)

Kaggle source here for competition: https://www.kaggle.com/competitions/nlp-getting-started

In [8]:
# Download data as zip
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-08-20 11:48:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.128, 142.251.31.128, 142.251.18.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.28MB/s    in 0.5s    

2023-08-20 11:48:57 (1.28 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [9]:
# Unzip data
zip_ref = zipfile.ZipFile('nlp_getting_started.zip')
zip_ref.extractall()
zip_ref.close()

## Becoming one with the data

We got the data and now, we need to understand our data that's wht lets visualize the text data.

In [10]:
# Read the data and turn it pandas dataframe
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [11]:
# Check out the first 5 row
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
# Get the info from train dataset
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [13]:
# Check the text row
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [14]:
# Shuffle train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [15]:
# Check the shuffled data
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [16]:
# How does look like test data?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [17]:
# How many examples of each class?
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [18]:
# How many total samples?
print(f'Train total samples: {len(train_df_shuffled)}')
print(f'Test total samples: {len(test_df)}')

Train total samples: 7613
Test total samples: 3263


In [19]:
# Let's visualize the some random training examples
random_samples = random.sample(range(0, len(train_df_shuffled)), 5)

for i in random_samples:
    target = train_df_shuffled['target'][i]
    text = train_df_shuffled['text'][i]
    print(f'Target: {target}', '(Real Diaster)' if target == 1 else '(Not Real Diaster)')
    print(f'Text: {text}', '\n')

Target: 0 (Not Real Diaster)
Text: The worst  voice I can ever hear is the 'Nikki your in trouble' voice from my mom 

Target: 0 (Not Real Diaster)
Text: Fucking yes /r/antiPOZi is quarantined.  Triggered the cucks we have. 

Target: 1 (Real Diaster)
Text: 2pcs 18W CREE Led Work Light  Offroad Lamp Car Truck Boat Mining 4WD FLOOD BEAM - Full reaÛ_ http://t.co/ApWXS5Mm44 http://t.co/DS76loZLSu 

Target: 0 (Not Real Diaster)
Text: Watch Sarah Palin OBLITERATE Planned Parenthood For Targeting Minority Women! ÛÒ BB4SP http://t.co/sAYZt2oagm 

Target: 1 (Real Diaster)
Text: If you're in search of powerful content to improve your business or have been frustrated with the deluge of 'quantitÛ_https://t.co/64cyMG1lTG 



### Split data into training and validation sets

In [20]:
# Use train_test_split to split our data
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [21]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [22]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before build a model is to convert text into the numbers.

There are few ways to do this:
* Tokenization - direct mapping of token (a token could be a word or a characher) to number
* Embedding - create a matrix of feature vektor for each token (the size of the feature vektor can be defined and this embedding can be learned)

### Text vektorization (tokenization)

TensorFlow Vectorization Docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [59]:
# Use the default text vektorization parameters
text_vektorizer = TextVectorization(max_tokens=None, # How many words in the sentence (automatically add <OOV> OOV:Out of word)
                                    standardize='lower_and_strip_punctuation', # lower case the word and delete punctuations
                                    split='whitespace', # Split the sentence with whitespace
                                    ngrams=None, # Create groups of n-words?
                                    output_mode='int', # How to map tokens to numbers
                                    output_sequence_length=None, # How long do you want your sequences to be?
                                    pad_to_max_tokens=False)

In [54]:
# Find the average number of tokens (words) in the training tweets
total_words = sum([len(i.split()) for i in train_sentences])
total_sentences = len(train_sentences)
round(total_words / total_sentences)

15

In [60]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see)

text_vektorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length,
                                    pad_to_max_tokens=True)

In [61]:
# Fit the text vectorizer to the training text
text_vektorizer.adapt(train_sentences)

In [65]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vektorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [69]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print('Random sentence:\n', random_sentence, '\n')
print('Vectorized version:\n', text_vektorizer([random_sentence]))

Random sentence:
 Tried orange aftershock today. My life will never be the same 

Vectorized version:
 tf.Tensor(
[[1055 3555  966  124   13  129   38  192   21    2  726    0    0    0
     0]], shape=(1, 15), dtype=int64)


In [72]:
# Get the unique words in the vocabulary
words_in_vocab = text_vektorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocabulary: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocabulary: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding layer

Tensorflow Embedding Docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we mostly care about the Embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [73]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                                      output_dim=128,
                                      input_length=max_length) # how long is each input
embedding

In [88]:
# Get a random sentence form the training set
random_sentence = random.choice(train_sentences)
print(f'Random sentence:\n', random_sentence, '\n')

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vektorizer([random_sentence]))
print(f'Embedding version:\n', sample_embed)

Random sentence:
 #NowPlaying School Of Seven Bells - Windstorm #WZBT 

Embedding version:
 tf.Tensor(
[[[ 0.03275803  0.00123842 -0.03384979 ...  0.03285385 -0.00440595
    0.04111486]
  [ 0.00796136  0.03821358 -0.0198498  ... -0.03847864 -0.0171295
    0.01356205]
  [-0.00576775  0.01805449 -0.03671279 ... -0.01109927  0.00925436
    0.04107213]
  ...
  [ 0.04982093  0.04987091  0.04211577 ... -0.0296942   0.01721479
    0.00821581]
  [ 0.04982093  0.04987091  0.04211577 ... -0.0296942   0.01721479
    0.00821581]
  [ 0.04982093  0.04987091  0.04211577 ... -0.0296942   0.01721479
    0.00821581]]], shape=(1, 15, 128), dtype=float32)


In [91]:
# Check out the single token's embedding
random_sentence, sample_embed[0][0], sample_embed[0][0].shape

('#NowPlaying School Of Seven Bells - Windstorm #WZBT',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03275803,  0.00123842, -0.03384979, -0.01517167,  0.01288921,
         0.01486235,  0.03514916,  0.0056209 ,  0.04627191, -0.04192974,
        -0.03089321, -0.01012842, -0.02331505, -0.00668811, -0.00034518,
        -0.04951527,  0.00796302, -0.00056503, -0.01037371,  0.01928145,
         0.03184174,  0.04974755,  0.03453544,  0.00906194,  0.01554671,
        -0.0215704 ,  0.02309113,  0.00442587, -0.02937472, -0.04640638,
         0.0104482 , -0.02138534,  0.02731062, -0.0450692 ,  0.00344324,
        -0.00330647,  0.04290321, -0.04734069,  0.01042347,  0.0104575 ,
        -0.03573962, -0.04747843,  0.00393729,  0.00878433,  0.04327593,
         0.02349857, -0.01990216, -0.04051595, -0.02818309,  0.00616487,
        -0.00593337, -0.00065707, -0.04347925,  0.00429996,  0.02483359,
        -0.03260701, -0.01122023,  0.03807122, -0.01906198,  0.03740491,
        -0.00305356

## Modelling a text dataset (running a series of experiments)

We converted our text data into numbers with vectorization and embedding, now it is time to create models. Here are the models we will create step by step:

* Model 0: [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) with TF-IDF encoder (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional neural network
* Model 6: TensorFlow Hub pretrained feature extractor
* Model 7: Same as model 6 with 10% of training data

Standard steps in the modelling with tensorflow:
* Create a model
* Compile the model
* Fit the mdel
* Evaluate the model
* Improve through experimentation

### Modeling 0: Getting a baseline (Naive Bayes with TF-IDF)

Creating a baseline model is so important as with all machine learning modelling experiments for bencmarking.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to integers.

**Note:** It's so common practice to use non-DL(deep learning) algorithms as a baseline because of their speed. Later on, we can use DL algorithm and improve upon the non-DL models.

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # convert words to numbers using TF-IDF
    ('clf', MultinomialNB()) # model the text
])

# Fit the pipline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [105]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline model's accuracy: %{baseline_score*100:.2f}")

Baseline model's accuracy: %79.27


In [106]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
# First 20 predictions
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time such as:

* Accuracy
* Precision
* Recall
* F1-Score

Let's create a fuction to evaluate our model's prediction with above metrics.

In [107]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculate the model accuracy, precision, recall, f1-score of a binary classification model.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred)
    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    model_results = {
        'accuracy': model_accuracy,
        'precision': model_precision,
        'recall': model_recall,
        'f1': model_f1
    }
    return model_results

In [108]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Feed Forward Neural Network (Dense model)